# calculations

In [ ]:
library(terra)
library(parallel)
library(sf)
library(eurostat)
library(ggplot2)
library(tidyterra)  # Converts terra objects for ggplot


In [ ]:
path <- "../data/in"

In [ ]:
df_R0_Cpip_post <- read.csv(file.path("..", "data", "in", "R0_pip_stats.csv"))


In [ ]:
summary(df_R0_Cpip_post)

# load R0 pipiens posterior stats


In [ ]:
#temperature values at which statistics where derived
temps <- seq(0, 45, by = 0.1)
temps

In [ ]:
# Prebuild interpolation function
interp_fun <- approxfun(x = temps, y = df_R0_Cpip_post$Median_R0, 
                        method="linear", 
                        rule = 1)

In [ ]:
R0_pip_fun <- function(Tm){
  
  result <- rep(0, length(Tm))
  result[is.na(Tm)] <- NA
  
  valid <- !is.na(Tm) & Tm >= 0 & Tm <= 45
  
  result[valid] <- interp_fun(Tm[valid])
  
  return(result)
}


In [ ]:
eu_nuts3 <- get_eurostat_geospatial(resolution = 10,
                                    nuts_level = 3, 
                                    year = 2024) 

In [ ]:
eu_nuts3

In [ ]:
out_data = file.path("..", "data", "out")

# Daily data 

In [ ]:
year = 2024 
month = 8
target_file <- paste0("ERA5land_global_t2m_dailyStats_mean_01Deg_",
                        year,
                        "_",
                        sprintf("%02d", month),
                        "_data.nc")
target_file <- file.path(path, target_file)
r_tmp = terra::rast(target_file)
r_tmp


# Use model

In [ ]:
cl <- makeCluster(16)  
# export relevant functions to cluster
clusterExport(cl, 
              list("df_R0_Cpip_post",
                   "R0_pip_fun", 
                   "interp_fun"
)) 
r_R0 <- NULL
map_uncropped <- NULL
map_cropped <- NULL

for(year in 2024:2024){
  
  for(month in 8:8){
    target_file <- paste0("ERA5land_global_t2m_dailyStats_mean_01Deg_",
                          year,
                          "_",
                          sprintf("%02d", month),
                          "_data.nc")
    print(paste("month:", sprintf("%02d", month)))
    target_file <- file.path(path, target_file)
    # # create spatRaster object
    r_tmp = terra::rast(target_file)
    map_uncropped <<- r_tmp
    
    # crop to Europe extent
    r_tmp <- terra::crop(r_tmp, eu_nuts3)
    map_cropped <<- r_tmp 

    
    # apply R0 transformation
    r_R0 <<- app(r_tmp, 
              fun = R0_pip_fun,
              cores = cl)
    
    # # save output
    output_filename = paste0("R0_pip_posterior_median_europe_", year,"_", 
                             sprintf("%02d", month), ".tif")
    output_filename <- file.path(out_data, output_filename)
    writeRaster(r_R0, filename = output_filename,
                overwrite = TRUE)
    
    # print(paste(year,month))
    print(paste('target_file', target_file))
    print(paste('output_file', output_filename))
  }
}
stopCluster(cl)

# Inspec the results for the daily data

In [ ]:
r_R0

In [ ]:
nlyr(r_R0) # n layers in the raster

In [ ]:
map_uncropped_mean <- app(map_uncropped, fun = mean, na.rm = TRUE)

ggplot() +
  geom_spatraster(data = map_uncropped_mean) +
  scale_fill_viridis_c(name = "Temp (°C)", na.value = "transparent") +
  labs(title = "Temperature in Europe",
       subtitle = "August 2024") +
  theme_minimal()

In [ ]:
map_cropped_mean <- app(map_cropped, fun = mean, na.rm = TRUE)

ggplot() +
  geom_spatraster(data = map_cropped_mean) +
  scale_fill_viridis_c(name = "Temp (°C)", na.value = "transparent") +
  labs(title = "Temperature in Europe",
       subtitle = "August 2024") +
  theme_minimal()

In [ ]:
r_R0_mean <- app(r_R0, fun = mean, na.rm = FALSE) # this does something werid, the values that come out should be normalized

ggplot() +
  geom_spatraster(data = r_R0_mean) +
  scale_fill_viridis_c(name = "R0", na.value = "transparent") +
  labs(title = "Mosquito Reproductive Potential (R0) in Europe",
       subtitle = "August 2024") +
  theme_minimal()

# For monthly data

In [ ]:
target_file <- paste0("era5_data_2024_01_02_03_2t_tp_monthly_celsius_mm_resampled_05degree_trim.nc")
target_file <- file.path(path, target_file)
r_tmp = terra::rast(target_file)
r_tmp

In [ ]:
nlyr(r_tmp)

## Get temperature data and inspect

In [ ]:
r_tmp <- r_tmp[[c("t2m_1", "t2m_2", "t2m_3")]]

In [ ]:
r_mean <- app(r_tmp, fun = mean, na.rm = TRUE)

ggplot() +
  geom_spatraster(data = r_mean) +
  scale_fill_viridis_c(name = "Temp (°C)", na.value = "transparent") +
  labs(title = "Temperature ",
       subtitle = "mean Jan-March 2024") +
  theme_minimal()

In [ ]:
jan_temp <- r_tmp[["t2m_1"]]
ggplot() +
  geom_spatraster(data = jan_temp) +
  scale_fill_viridis_c(name = "Temp (°C)", na.value = "transparent") +
  labs(title = "Temperature ",
       subtitle = "Jan 2024") +
  theme_minimal()

## apply model to daily data

In [ ]:
cl <- makeCluster(16)  
# export relevant functions to cluster
clusterExport(cl, 
              list("df_R0_Cpip_post",
                   "R0_pip_fun", 
                   "interp_fun"
)) 

r_R0 <- NULL

# crop to Europe extent
r_tmp <- terra::crop(r_tmp, eu_nuts3)

# apply R0 transformation
r_R0 <<- app(r_tmp, 
            fun = R0_pip_fun,
            cores = cl)

# # save output
output_filename = paste0("R0_pip_posterior_median_europe_daily_2024.tif")
output_filename <- file.path(out_data, output_filename)
writeRaster(r_R0, filename = output_filename,
            overwrite = TRUE)

# print(paste(year,month))
print(paste('target_file', target_file))
print(paste('output_file', output_filename))

stopCluster(cl)

## Inspect results

In [ ]:
nlyr(r_R0) # n layers in the raster

In [ ]:
r_R0

In [ ]:
r_R0_mean <- app(r_R0, fun = mean, na.rm = FALSE) # this does something werid, the values that come out should be normalized

ggplot() +
  geom_spatraster(data = r_R0_mean) +
  scale_fill_viridis_c(name = "R0", na.value = "transparent") +
  labs(title = "R0",
       subtitle = "Mean January-March 2024") +
  theme_minimal()

In [ ]:
ggplot() +
  geom_spatraster(data = r_R0[["lyr.1"]]) +
  scale_fill_viridis_c(name = "R0", na.value = "transparent") +
  labs(title = "R0",
       subtitle = "January"
  )  +
  theme_minimal()